In [10]:
import logging
import time
import akshare as ak
import sqlalchemy as db  # Version Check
import pandas as pd
import logging
import sqlalchemy as db 
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData, Table, Sequence, Column, Float, Integer, String
from sqlalchemy.sql import select


In [11]:
def str2str(s, strp, strf):
    return time.strftime(
        strf, 
        time.strptime(
            s, 
            strp)
    )

In [12]:
connection_uri = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}".format(**{
    "user": "root",
    "password": "mysql123456",
    "host": "localhost",
    "port": "3306",
    "dbname": "xinguan"
})
engine = db.create_engine(connection_uri)


In [13]:
metadata = MetaData()

t_daily = Table(
    'csse_daily',
    metadata,
    Column('us_only_county_code_fips', Integer),
    Column('us_only_county_name', String(64)),
    Column('higher', String(64)),
    Column('lower', String(64)),
    Column('last_update', String(64)),
    Column('last_update_unify', String(64)),
    Column('report_day', String(64)),
    Column('latitude', Float),
    Column('longitude', Float),
    Column('confirmed', Integer),
    Column('dead', Integer),
    Column('recovered', Integer),
    Column('active', Integer),
    Column('district', String(64)),
)

t_daily.drop(engine, checkfirst=True)
t_daily.create(engine)


In [15]:
retry = 100
report_day = "2020-01-22"
for j in range(retry):
    try:
        df0122 = ak.covid_19_csse_daily(date=report_day)
        logging.info("成功WEB：{0}".format(report_day))
        print("成功WEB：{0}".format(report_day))
        time.sleep(0.1)
        break
    except Exception as e:
        if j == retry - 1:
            logging.info(e)
            logging.info("失败WEB：{0}".format(report_day))
            print("失败WEB：{0}".format(report_day))
        logging.info(e)
        logging.info("重试WEB：{0}".format(report_day))
        print("重试WEB：{0}".format(report_day))

重试WEB：2020-01-22
重试WEB：2020-01-22
成功WEB：2020-01-22


In [16]:
df0122.rename(columns={
    'Province/State':'higher',
    'Country/Region':'lower',
    'Last Update':'last_update',
    'Confirmed':'confirmed',
    'Deaths':'dead',
    'Recovered':'recovered'
}, inplace=True)


In [17]:
'%Y-%m-%d'
'%m/%d/%y %H:%M'
'%Y-%m-%dT%H:%M:%S'

df0122['last_update_unify'] = df0122['last_update'].map(lambda x:str2str(x, '%m/%d/%Y %H:%M', '%Y-%m-%dT%H:%M:%S'))     
# df0122['report_day'] = df0122['last_update'].map(lambda x:str2str(x, '%m/%d/%Y %H:%M', '%Y-%m-%d')) 
df0122['report_day'] = report_day


In [18]:
df0122.fillna(value={
    'higher':'null',
    'lower':'null',
    'confirmed':0,
    'dead':0,
    'recovered':0,
}, inplace=True)
df0122

,higher,lower,last_update,confirmed,dead,recovered,last_update_unify,report_day
0,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
1,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
3,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
4,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
...,...,...,...,...,...,...,...,...
33,Yunnan,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
34,Zhejiang,Mainland China,1/22/2020 17:00,10.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
35,null,Japan,1/22/2020 17:00,2.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22
36,null,Thailand,1/22/2020 17:00,2.0,0.0,0.0,2020-01-22T17:00:00,2020-01-22


In [19]:
for j in range(retry):
    try:
        df0122.to_sql(
            t_daily.name,
            con=engine,
            if_exists='append',
            index=False)
        logging.info("成功DB：{0}".format(report_day))
        print("成功DB：{0}".format(report_day))
        time.sleep(0.1)
        break
    except Exception as e:
        if j == retry - 1:
            logging.info("失败DB：{0}".format(report_day))
            print("失败DB：{0}".format(report_day))
        logging.info("重试DB：{0}".format(report_day))
        print("重试DB：{0}".format(report_day))
        logging.info(e)

成功DB：2020-01-22


* `>= 01-22-2020.csv`
* `>= 03-01-2020.csv`
* `>= 03-22-2020.csv`

***


* FIPS: US only. FederalInformationProcessingStandards. Code that uniquely identifies counties within the USA.
* Admin2: US only. County name. 
* Province_State: Province, state or dependency name. 
    * Province/State: 
        * China - province name; 
        * US/Canada/Australia/ - city name, state/province name; 
        * Others - name of the event (e.g., "Diamond Princess" cruise ship); 
        * other countries - blank.
* Country_Region: 
    * Country, region or sovereignty name. The names of locations included on the Website correspond with the official designations used by the U.S. Department of State.
* Last Update: MM/DD/YYYY HH:mm:ss (24 hour format, in UTC).
* Lat and Long_: Dot locations on the dashboard. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.
* Confirmed: Confirmed cases include `presumptive positive` cases and `probable` cases, in accordance with CDC guidelines as of April 14.
* Deaths: Death totals in the US include confirmed and probable, in accordance with CDC guidelines as of April 14.
* Recovered: Recovered cases outside China are estimates based on local media reports, and state and local reporting when available, and therefore may be substantially lower than the true number. US state-level recovered cases are from COVID Tracking Project.
* Active: Active cases = total confirmed - total recovered - total deaths.
* Incidence_Rate: Admin2 + Province_State + Country_Region.
* Case-Fatality Ratio (%): = confirmed cases per 100,000 persons.
* US Testing Rate: = total test results per 100,000 persons. The "total test results" is equal to "Total test results (Positive + Negative)" from COVID Tracking Project.
* US Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases. The "Total number hospitalized" is the "Hospitalized – Cumulative" count from COVID Tracking Project. The "hospitalization rate" and "hospitalized - Cumulative" data is only presented for those states which provide cumulative hospital data.

***

Update frequency:
* Files on and after April 23, once per day between 03:30 and 04:00 UTC.
* Files from February 2 to April 22: once per day around 23:59 UTC.
* Files on and before February 1: the last updated files before 23:59 UTC. Sources: archived_data and dashboard.

test


tupleest
